In [5]:
from flask import Flask, request, render_template, jsonify
import pandas as pd
import numpy as np
import re
import csv
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
app = Flask(__name__, template_folder='templateFiles')

## API Untuk Input Text ##

In [7]:
@app.route('/')
def my_form():
    return render_template('text_sentiment.html')

In [8]:
@app.route('/', methods=['POST'])
def my_form_post():
    text = request.form['text']
    tknzr = pickle.load(open('tokenizer_lstm.pkl','rb'))
    model = pickle.load(open('model_lstm.pkl', 'rb'))

    text = text.replace("\\n", " ")
    text = re.sub(r"(#)(.*)()", r"", text)
    text = re.sub(r"[!\"#$%&'()*+-./:;<=>?@[\]^_`{|}~]", r"", text)
    text = re.sub(r"(\s)(\1+)", r"\1", text)
    text = text.replace("URL","")
    text = text.replace("RT ","")
    text = text.replace("USER ","")
    text = text.replace("USER","")
    text = re.sub(r"([a-z])(\1{3,})", r"\1\1", text)
    text = re.sub(r"(\\x)([a-z0-9]{2})", r"", text)
    text  = re.sub(r"(\\x8)", r"", text)
    text = text.lower()
    text = text.strip()
    
    preprocess_text = text
    max_features = 100
    
    new_X_val = [preprocess_text]
    new_X_val = tknzr.texts_to_sequences(new_X_val)
    new_X_val = pad_sequences(new_X_val, maxlen=max_features)
    new_y_pred = model.predict(new_X_val)
    new_y_pred = np.argmax(new_y_pred, axis=1)
    new_y_pred = new_y_pred.tolist()
    #new_y_pred = lb.inverse_transform(new_y_pred)
    
    sentiment = []
    for code in new_y_pred:
        if code == 0:
            sentiment.append("Negative Sentiment")
        elif code == 1:
            sentiment.append("Neutral Sentiment")
        else:
            sentiment.append("Positive Sentiment")

    response = {
    'data' : preprocess_text,
    'result sentiment' : sentiment
    }
    
    response_data = response
    return response_data

## API Untuk Input File

In [9]:
@app.route('/csv-upload', methods=['GET', 'POST'])
def index():
    return render_template('index.html')

In [10]:
def preprocessing_text(text): 
    text = text.replace("\\n", " ")
    text = re.sub(r"(#)(.*)()", r"", text)
    text = re.sub(r"[!\"#$%&'()*+-./:;<=>?@[\]^_`{|}~]", r"", text)
    text = re.sub(r"(\s)(\1+)", r"\1", text)
    text = text.replace("URL","")
    text = text.replace("RT ","")
    text = text.replace("USER ","")
    text = text.replace("USER","")
    text = re.sub(r"([a-z])(\1{3,})", r"\1\1", text)
    text = re.sub(r"(\\x)([a-z0-9]{2})", r"", text)
    text  = re.sub(r"(\\x8)", r"", text)
    text = text.lower()
    text = text.strip()
    return text

In [11]:
tknzr = pickle.load(open('tokenizer_lstm.pkl','rb'))
model = pickle.load(open('model_lstm.pkl', 'rb'))
max_features = 100

def input_data(text):
    new_X_val = [text]
    new_X_val = tknzr.texts_to_sequences(new_X_val)
    new_X_val = pad_sequences(new_X_val, maxlen=max_features)
    new_y_pred = model.predict(new_X_val)
    new_y_pred = np.argmax(new_y_pred, axis=1)
    new_y_pred = new_y_pred.tolist()
    return new_y_pred

In [12]:
@app.route('/data', methods=['GET', 'POST'])
def data():
    if request.method == 'POST':
        file = request.form.get('csvfile')
        data = []
        data = pd.read_csv(file)
        
        data['process_text'] = data['Tweet'].apply(lambda x:preprocessing_text(x))
        data['sentiment'] = data['process_text'].apply(lambda x:input_data(x))
            
        predict = []
        for code in data['sentiment']:
            if code == [0]:
                predict.append("Negative Sentiment")
            elif code == [1]:
                predict.append("Neutral Sentiment")
            else:
                predict.append("Positive Sentiment")
        
        data['predict'] = predict
        
        data = data[['Tweet', 'process_text', 'predict']]
        
        return render_template('data.html', data=data.to_html(header=True, index=False))

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
